<h1><center> Предобработка данных Web of Science </center></h1>

<h2><center> Часть 1. Загрузка данных </center></h2>

In [291]:
import pandas as pd
import numpy as np
import tqdm

columns = ["id", "author", "documents", "citations", "total link strength"]

with open('Все авторы.txt') as file:
    lines = [i.strip() for i in file.readlines()]

In [292]:
indexies = [i.split('\t')[0] for i in lines]
indexies.pop(0)
authors = [i.split('\t')[1] for i in lines]
authors.pop(0)
documents = [i.split('\t')[2] for i in lines]
documents.pop(0)
citations = [i.split('\t')[3] for i in lines]
citations.pop(0)
totals = [i.split('\t')[4] for i in lines]
totals.pop(0)
print()

In [293]:
df = pd.DataFrame([indexies, authors, documents, citations, totals]).transpose()
df.columns = columns
df['author'] = df['author'].apply(lambda x: x[1:-1])
df.index = [int(i) for i in df['id']]
df = df.drop(axis=1, columns=['id'])
df['documents'] = df['documents'].apply(int)
df['citations'] = df['citations'].apply(int)
df['total link strength'] = df['total link strength'].apply(int)
df # итоговый датафрейм

,author,documents,citations,total link strength
1,"abaeva, a. m.",1,1,4
2,"abaeva, asiyat m.",1,0,4
3,"abaeva, o. p.",2,0,8
4,"abalakin, a. a.",1,0,3
5,"abalakina, t. v.",1,0,3
...,...,...,...,...
8533,"zyrianov, v. v.",1,0,3
8534,"zyryanov, v. v.",2,3,3
8535,"zyryanov, vladimir v.",1,0,2
8536,"zyuganov, ga",1,0,20


In [294]:
old_author_names = df['author']

In [295]:
df.index # индексы начинаются с 1! как в изначальном файле -- обратите внимание!

Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
       ...
       8528, 8529, 8530, 8531, 8532, 8533, 8534, 8535, 8536, 8537],
      dtype='int64', length=8537)

In [296]:
range(1, df['author'].shape[0]+1) # правильный рэндж

range(1, 8538)

In [297]:
def get_unique_first_names_and_patronymics_combinations(df, fullname_column, index_starts=1):
    '''
    Возвращает pandas.DataFrame с подсчетом встречаемости уникальных комбинаций имен-отчеств*
    _________________
    * при этом под такими комбинациями считается то, что встречается после запятой в данных WOS.
    Пример:
        Иванов, А.Б. -> А.Б.
        Петров, Алексей И. -> Алексей, И.
        Петров -> None (если нет запятой, не учитывается в дальнейших подсчетах)
    __________________
            Параметры:
                    df (DataFrame): датафрейм с данными WOC;
                    fullname_column (string): название колонки, где содержатся полные имена авторов;
                    index_starts (int): 1 по умолчанию; цифра, с которой начинается нумерация индексов (0 или 1);

            Возвращает:
                    first_names_df (pandas.DataFrame): датафрейм с информацией 
                    
            Пример:
                    names_and_patronymics = get_unique_first_names_and_patronymics_combinations(df, fullname_column='author', index_starts=1)
                    names_and_patronymics # результат
    '''
    
    first_names_dict = {} # словарь для подсчета имен-отчеств
    for index in range(index_starts, df[fullname_column].shape[0] + 1):
        fullname = df.loc[index, fullname_column]
        if ',' in fullname:
            fullname_list = fullname.split(', ') # тут у нас полный список: фамилия, имя отчество
            first_name = "".join(fullname_list[1:]).strip() # тут у нас имя - все что после ПЕРВОЙ запятой!
            first_names_dict[first_name] = first_names_dict.get(first_name, 0) + 1 # обновляем словарь
    # Создаем датафрейм и выводим его
    first_names_df = pd.DataFrame([first_names_dict])
    first_names_df = first_names_df.transpose().reset_index()
    first_names_df.columns = ['author', 'n']
    
    return first_names_df

# Итак, изначально у нас 2508 комбинации имен, имен-отчеств в полной или неполной форме
names_and_patronymics = get_unique_first_names_and_patronymics_combinations(df, fullname_column='author', index_starts=1)
names_and_patronymics

,author,n
0,a. m.,26
1,asiyat m.,1
2,o. p.,6
3,a. a.,79
4,t. v.,23
...,...,...
2503,v. ye.,1
2504,natalya vasilevna,1
2505,n. z.,1
2506,dennis,1


In [298]:
names_and_patronymics.sort_values('n', ascending=False).head(20) # топ-20 самых популярных

,author,n
52,v. v.,90
152,a. v.,85
3,a. a.,79
55,vv,78
121,a.v,72
68,e. a.,64
134,av,56
153,v. a.,56
36,m. a.,55
140,va,52


In [299]:
def get_unique_last_names(df, fullname_column, index_starts=1):
    '''
    Возвращает pandas.DataFrame с подсчетом встречаемости уникальных фамилий авторов*
    _________________
    * при этом под фамилиями считается то, что встречается до запятой в данных WOS.
    Пример:
        Иванов, А.Б. -> Иванов
        Петров, Алексей И. -> Петров
        Петров -> Петров
        Елена, Александровна Рождественская -> Елена
    __________________
            Параметры:
                    df (DataFrame): датафрейм с данными WOC;
                    fullname_column (string): название колонки, где содержатся полные имена авторов;
                    index_starts (int): 1 по умолчанию; цифра, с которой начинается нумерация индексов (0 или 1);

            Возвращает:
                    last_names_df (pandas.DataFrame): датафрейм с информацией 
                    
            Пример:
                    last_names = get_unique_last_names(df, fullname_column='author', index_starts=1)
                    last_names # результат
                
    '''
    last_names_dict = {}
    # Подсчет фамилий
    for index in range(index_starts, df[fullname_column].shape[0]+1):
        fullname = df.loc[index, 'author'] # тут у нас имя автора;
        fullname_list = fullname.split(', ') # тут у нас полный список: фамилия, имя отчество
        last_name = fullname_list[0]
        last_names_dict[last_name] = last_names_dict.get(last_name, 0) + 1
 
    # Создание датафрейма
    last_names_df = pd.DataFrame([last_names_dict])
    last_names_df = last_names_df.transpose().reset_index()
    last_names_df.columns = ['author', 'n']
    return last_names_df

last_names = get_unique_last_names(df, fullname_column='author', index_starts=1)
last_names # до обработки 5807 уникальных фамилий

,author,n
0,abaeva,3
1,abalakin,1
2,abalakina,1
3,abalkin,1
4,abashidze,1
...,...,...
5802,zykovskaya,1
5803,zyrianov,1
5804,zyryanov,2
5805,zyuganov,1


In [300]:
last_names.sort_values('n').tail(10) # самые популярные

,author,n
4039,romanov,13
5416,volkov,15
4317,semenov,15
1866,ivanov,16
4700,smirnov,16
4701,smirnova,17
3844,popov,18
4722,sokolov,19
3845,popova,26
1867,ivanova,29


<h2><center>Часть 2. Диагностика проблем и их решение</center></h2>

**Просмотрев файл с данными, мне показалось, что логично будет работать так:**
1. Предобработать данные;
2. Поработать с порядком фамилий, имен и отчеств;
3. Привести максимально схожие ФИО к единой форме с помощью расстояния Левенштейна.

**1. Предобработать данные**

1.1 Убрать штрих `'` из всех ФИО (иногда встречаются такие фамилии; пример – `avksent'ev, v. a.`, хотя есть аналогичные без мягкого знака);

1.2 Заменить написания некоторых имен на канонические (`yekaterina` --> `ekaterina`).

In [301]:
# Еще одна проблема - не во всех строках вообще есть запятые; есть экзотические наименования
print(f'В изначальном датафрейме есть следующие полные имена авторов, в которых нет запятой ->')
print('То есть, даже потенциального указания на деление на имя и фамилию; а именно:\n')
problem_index = []
for index in range(1, df['author'].shape[0]+1):
    fullname = df.loc[index, 'author']
    fullname_list = fullname.split(', ') # тут у нас полный список: фамилия, имя отчество
    
    try:
        name_and_patronymic = fullname_list[1] # получаем имя и отчество
    except:
        print(fullname)
        problem_index.append(index)
print('\nИх индексы: ')
print(problem_index) # тут сохранены эти индексы

В изначальном датафрейме есть следующие полные имена авторов, в которых нет запятой ->
То есть, даже потенциального указания на деление на имя и фамилию; а именно:

ai haidon
aldinovaelzaochir-goryaevn
ieu minh v
iang haibi
engka
iao xume
nstan
un feiha
ang zhengxian
ei ruozh
urengaow
ang chunle
e chonglian
hao tiantia

Их индексы: 
[1241, 1243, 2409, 2695, 4484, 4528, 5113, 7138, 7212, 8034, 8052, 8103, 8160, 8387]


In [302]:
# 1.1 Убрать штрих `'` из всех ФИО;
# Пример: есть 425 "avksent'ev, v. a." а может стать '"avksentiev, v. a."'
print(f'До обработки: {last_names.shape[0]} уникальных фамилий;')
print(f'До обработки: {names_and_patronymics.shape[0]} уникальных комбинаций имен-отчеств;')

df['author'] = df['author'].apply(lambda x: x.replace("'", ""))

last_names = get_unique_last_names(df, fullname_column='author', index_starts=1)
names_and_patronymics = get_unique_first_names_and_patronymics_combinations(df, fullname_column='author', index_starts=1)

print(f'После обработки обработки: {last_names.shape[0]} уникальных фамилий;')
print(f'После обработки: {names_and_patronymics.shape[0]} уникальных комбинаций имен-отчеств;')

До обработки: 5807 уникальных фамилий;
До обработки: 2508 уникальных комбинаций имен-отчеств;
После обработки обработки: 5768 уникальных фамилий;
После обработки: 2503 уникальных комбинаций имен-отчеств;


In [303]:
# 1.2 Заменить написания некоторых имен на канонические;
# тут ключ - неканоническое написание; значение - каноническое (то на которое будем менять)
# так, мы будем менять audrey НА andrey
# Надо понимать, что этот список можно расширять/сокращать при желании, и он не закончен;
# В файле Все_авторы.txt я встретила другие странные записи: lalita, youri, zhu-liana итд

# Уже некоторое время спустя я обнаружила что существует ГОСТ 7.79-2000, который реализован в коде
# модуля transliterate для транслитерации с/на русский язык: https://docs-python.ru/packages/modul-transliterate-python/
# это выглядит так: 'Михаил, Юлия, София, Андрей' ----> 'Mihail, Julija, Sofija, Andrej'

names_dict = {"audrey" : "andrey",
    "andrei": "andrey",
    "andrej" : "andrey",
    "anatoli": "anatoliy",
    "anatolii": "anatoliy",
    "aleksandr": "alexander",
    "alexandr": "alexander",
    "alexsander": "alexander",
    "aleksander": "alexander",
    "alekxandr": "alexander",
    "aleksandra": "alexandra",
    "anastasiya" : "anastasia",
    "anastassia": "anastasia",
    "anastassiya": "anastasia",
    "angelika": "anzhelika",
    "alexei": "alexey",
    "aleksei" : "alexey",
    "andzhela": "anzhela",
    "anatolii": "anatoliy",
    "anatoly": "anatoliy",
    "arkady": "arkadiy",
    "artyom": "artem",
    "arsenii": "arseniy",
    "arseny": "arseniy",
    "arseni": "arseniy",
    "darya" : "daria",
    "dennis": "denis",
    "dmitrij": "dmitriy",
    "dmitri" : "dmitriy",
    "dmitrii" : "dmitriy",
    "dmitry" : "dmitriy",
    "yegor": "egor",
    "elisaveta": "elizaveta",
    "evgeni": "evgeniy",
    "eugene": "evgeniy",
    "evgenii": "evgeniy",
    "eugueni" : "evgeniy",
    "eugeny": "evgeniy",
    "eugeniya" : "evgeniya",
    "evgeniia": "evgeniya",
    "helena" : "elena",
    "helen": "elena",
    "katerina": "ekaterina",
    "yekaterina": "ekaterina",
    "grigori": "grigoriy",
    "grigorii": "grigoriy",
    "grigory": "grigoriy",
    "georgii": "georgiy",
    "georgy": "georgiy",
    "gregory": "grigoriy",
    "gennadii": "gennadiy",
    "gennady": "gennadiy",
    "genadiy": "gennadiy",
    "genadii": "gennadiy",
    "ilia": "ilya",
    "iryna" : "irina",
    "camilla": "kamilla",
    "kammila": "kamilla",
    "ksenya": "ksenia",
    "kseniya": "ksenia",
    "constantin": "konstantin",
    "larissa": "larisa",
    "lillia": "lilia",
    "liliya": "lilia",
    "liliia": "lilia",
    "liudmila" : "ludmila",
    "lyudmila" : "ludmila",
    "lidia": "lidiya",
    "lidiia": "lidiya",
    "lyda": "lidiya",
    "liubov" : "lubov",
    "lyubov" : "lubov",
    "luisa": "luiza",
    "mariia" : "maria",
    "mariya": "maria",
    "maksim": "maxim",
    "mayya": "maya",
    "maiya": "maya",
    "marharita": "margarita",
    "marharyta": "margarita",
    "matthew": "matvey",
    "matvei": "matvey",
    "matveii": "matvey",
    "nikolai": "nikolay",
    "nadejda": "nadezhda",
    "nadezda": "nadezhda",
    "natalya" : "natalia",
    "nataliya": "natalia",
    "natalyia": "natalia",
    "nikolai": "nikolay",
    "nikitoy": "nikita",
    "olesya": "olesia",
    "olessia": "olesia",
    "oxana": "oksana",
    "volha": "olga",
    "pauline" : "polina",
    "pawel": "pavel",
    "serge": "sergey",
    "sergei" : "sergey",
    "serguey": "sergey",
    "shezana": "shezhana",
    "sofya": "sofia",
    "sofiya": "sofia",
    "sophia": "sofia",
    "sviatoslav": "svyatoslav",
    "tatyana" : "tatiana",
    "tatiyana" : "tatiana",
    "tatjyana": "tatiana",
    "timofei": "timofey",
    "timofeii": "timofey",
    "vasily": "vasiliy",
    "vasilii": "vasiliy",
    "viktor": "victor",
    "viktar": "victor",
    "viktoria": "victoria",
    "viktoriya": "victoria",
    "vitaly": "vitaliy",
    "vitali": "vitaliy",
    "vitalii": "vitaliy",
    "viacheslav": "vyacheslav",
    "valeri": "valeriy",
    "valerij": "valeriy",
    "valerii": "valeriy",
    "valeriya": "valeria",
    "yanya": "yana",
    "iana": "yana",
    "iuliia": "yulia",
    "julia": "yulia",
    "yuliya": "yulia",
    "yury": "yuriy",
    "yuri": "yuriy",
    "yurii": "yuriy",
    "uiriy": "yuriy",
    "uirii": "yuriy",
    "youri": "yuriy",
    "uliana": "ulyana",
    "jan": "yan",
    "zoia": "zoya",
    "zoiya": "zoya"}
# alisa
# victoria
# diana
# katerina
# yakov
# katia
# lisa
# liza
# andre
# bela
# nastia
# liana
# nadya
# katja
# taisiia
# katherine
# philipp
# filipp
# katharina
# nilya
# lelya
# inga
# inha
# nadia
# teodor
# theodor
# georg
# andrew
# susanna
# rita
# nastia
# liana


сounter = 0

print(f'До обработки: {last_names.shape[0]} уникальных фамилий;')
print(f'До обработки: {names_and_patronymics.shape[0]} уникальных комбинаций имен-отчеств;\n\n')

for index in range(1, df['author'].shape[0]+1):
    fullname = df.loc[index, 'author']
    if ',' in fullname:
        fullname_list = fullname.split(', ') # тут у нас полный список: фамилия, имя отчество
        name_and_patronymic = fullname_list[1] # получаем имя и отчество
        for one_part in name_and_patronymic.split():
            # тут меняем старое на новое!
            if one_part in names_dict:
                new_string = fullname.replace(one_part, names_dict[one_part])
                df.loc[index, 'author'] = new_string
                сounter += 1
                    
print(f'Было произведено {сounter} замен/ы', end='\n\n')

last_names = get_unique_last_names(df, fullname_column='author', index_starts=1)
names_and_patronymics = get_unique_first_names_and_patronymics_combinations(df, fullname_column='author', index_starts=1)
print(f'После обработки обработки: {last_names.shape[0]} уникальных фамилий;')
print(f'После обработки: {names_and_patronymics.shape[0]} уникальных комбинаций имен-отчеств;')

До обработки: 5768 уникальных фамилий;
До обработки: 2503 уникальных комбинаций имен-отчеств;


Было произведено 417 замен/ы

После обработки обработки: 5768 уникальных фамилий;
После обработки: 2383 уникальных комбинаций имен-отчеств;


In [304]:
# ПРОВЕРКА 1.2 - ничего не напечаталось, и не должно!
for index in range(1, df['author'].shape[0]+1):
    fullname = df.loc[index, 'author']
    if ',' in fullname:
        fullname_list = fullname.split(', ') # тут у нас полный список: фамилия, имя отчество
        name_and_patronymic = fullname_list[1] # получаем имя и отчество
        for one_part in name_and_patronymic.split():
            # тут меняем старое на новое!
            if one_part in names_dict:
                print(fullname)
                
# ничего не напечатано - все супер!

<b>2. Поработать с порядком фамилий, имен и отчеств</b>

Насколько мне представляется сейчас, наиболее корректным будет порядок записи следующего характера:

<code>фамилия, и. о.</code>

Такой порядок записи в данных представляется мне наиболее корректным, хотя встречаются разные форматы записи:

```
4763	"musaevna, idigova lalita"
4764	"museibov, ag"
4765	"mushaev, v. n."
4766	"musikhin, gleb"
4767	"musiychuk, m., v"	
4768	"musiychuk, s., v"	
4769	"muskhanova, i. v."
4770	"muss, alexander"
4771	"mussina, r. n."
4772	"mustaeva, f. a."
```</Python>

В списке ниже хочу зафиксировать, какие случаи несоответствия я зафиксировала:

**2.1** В некоторых случаях фамилии встречается еще одна запятая (`122	"aksiumov, b., v"`) -> убираем все запятые кроме первой;

**2.2** В некоторых случаях вперед выносится то, что является отчеством; однако, достаточно хорошо детектится по окончанию (`ovna`, `evna`, `vich`, `vith`) (`4763	"musaevna, idigova lalita"`) -> идем циклом, проверяем окончание "фамилий" (то, что на первом месте) на эти окончания, при несовпадении - меняем порядок;

**2.3** В некоторых случаях на первом месте находится не фамилия, а инициалы (и без точек, деления) (`213	"an, ershov"`) -> проходимся циклом, делаем сравнения по длине части до запятой и части после запятой; наверняка таких случаев будет не очень много и перестановку местами (с последующим разбиением инициалов на две части) можно будет использовать как решение.

**2.4** В некоторых случаях на первом месте находится полное имя, а на втором - фамилия (`216	"anastasiya, varzakova"`) -> нужно составить словарь с частотой встречаемости фамилий и имен; и если то что на первом месте имеет большУю встречаемость в словаре с частотностью имен, то поменять первое и второе местами.


**Помимо этого, нужно учитывать, что в русском языке есть буквы, которые не сократить до одного инициала**. Я выношу их сюда потому что посмотрела данные и поняла, что как минимум в каких-то случаях они так транслитерируются (но во всех ли случаях - вопрос); и с этим нужно что-то делать. 
* ш -> sh
* ж -> zh
* ч -> ch
* ю -> ju (`340	"artamonova, ju d.`)
* ю -> yu (`1336	"demidenko, svetlana yu.`)
* х -> kh (`azieva, raisa husainovna`)
* x -> h  (`aziyeva, r. kh.`)

**Похожего рода проблемы встречаются и с фамилиями. Встречаются почти полные копии одних и тех же фамилий**:
```
313	"archangelskiy, v. n."
314	"archangelsky, v. n."
327	"arkhangel'skiy, v. n."
328	"arkhangelskii, vn"
```

Или падежные формы имени автора: `254	"andriyashinym, nikitoy"`. К сожалению, в этом конкретном кейсе нет другой формы записи для этого автора, поэтому эта проблема останется нерешенной.

Думаю, лучше всего эти проблемы решатся с помощью дезамбигуации авторов по расстоянию Левенштейна.

**Но - последний шаг в этой части, это шаг 2.5**

**2.5** Приведение записей к единой форме `фамилия, и. о.`. На предыдущих этапах мы поработали с корректным порядком записи имен, фамилий и отчеств, и как мне кажется, теперь достаточно спокойно можем сократить изначальные строки до корректных записей.

In [305]:
# 2.1 После фамилии встречается еще одна запятая (`122 "aksiumov, b., v"`) 
# -> убираем все запятые кроме первой
# ПРИМЕР: abylkalikov, s., i -------> abylkalikov,  s. i.
сounter = 0

print(f'До обработки: {last_names.shape[0]} уникальных фамилий;')
print(f'До обработки: {names_and_patronymics.shape[0]} уникальных комбинаций имен-отчеств;\n\n')

for index in range(1, df['author'].shape[0]+1):
    fullname = df.loc[index, 'author']
    # Если у нас больше двух запятых, то
    if ',' in fullname and fullname.count(',') > 1:
        fullname_list = fullname.split(',') # тут у нас полный список: фамилия, имя отчество
        index_of_first_comma = fullname.index(',') # индекс первой заапятой
        text_after_first_comma_without_additional_comma = fullname[index_of_first_comma:].replace(',', '')
        fullname_correct = f'{fullname_list[0]}, {text_after_first_comma_without_additional_comma}.'
        df.loc[index, 'author'] = fullname_correct
        сounter += 1

print(f'Исправлено случаев: {сounter}', end='\n\n')

last_names = get_unique_last_names(df, fullname_column='author', index_starts=1)
names_and_patronymics = get_unique_first_names_and_patronymics_combinations(df, fullname_column='author', index_starts=1)
print(f'После обработки обработки: {last_names.shape[0]} уникальных фамилий;')
print(f'После обработки: {names_and_patronymics.shape[0]} уникальных комбинаций имен-отчеств;')

До обработки: 5768 уникальных фамилий;
До обработки: 2383 уникальных комбинаций имен-отчеств;


Исправлено случаев: 531

После обработки обработки: 5768 уникальных фамилий;
После обработки: 2311 уникальных комбинаций имен-отчеств;


In [306]:
# 2.1 ПРОВЕРКА: ничего не напечаталось (и не должно!)
for index in range(1, df['author'].shape[0]+1):
    fullname = df.loc[index, 'author']
    if ',' in fullname and fullname.count(',') > 1:
        print(fullname)
# ничего не напечаталось -> все ок

In [307]:
# 2.2 В некоторых случаях вперед выносится то, что является отчеством; однако, достаточно хорошо детектится по окончанию (`ovna`, `evna`, `vich`, `vith`) (`4763	"musaevna, idigova lalita"`) 
# -> идем циклом, проверяем окончание "фамилий" (то, что на первом месте) на эти окончания, при несовпадении - меняем порядок;
# ПРИМЕРЫ: 
# anatolyevna, kapustina valeria ---> kapustina, valeria anatolyevna
# andreevna, demyanova anna --------> demyanova, anna andreevna
# andrianovna, shabanova marina ----> shabanova, marina andrianovna
# ОДНАКО возникают беларусы...
# zubarevich, natalia vasilevna
# Появляется работа с исключением "Случай-исключение 1", "Случай-исключение 2"

counter = 0
patronymic_endings = ['evna', 'ovna', 'vich', 'vith']

print(f'До обработки: {last_names.shape[0]} уникальных фамилий;')
print(f'До обработки: {names_and_patronymics.shape[0]} уникальных комбинаций имен-отчеств;\n\n')

for index in range(1, df['author'].shape[0]+1):
    fullname = df.loc[index, 'author'] # тут у нас имя автора;
    if ',' in fullname:
        fullname_list = fullname.split(',') # тут у нас полный список: фамилия, имя отчество
        first_part_of_name = fullname_list[0]

        # теперь наша задача проверить, оканчивается ли "фамилия" на окончания ovna, evna, vich, vith
        # И при этом у нас всего три подстроки в этом полном имени 
        last_four_letters = first_part_of_name[-4:]
        if last_four_letters in patronymic_endings and len(fullname.split()) == 3:

            # Случай-исключение 1:
            all_text_after_first_comma_without_points = fullname[fullname.index(', '):].strip().replace('.', '').split()
            all_text_after_first_comma_without_points.remove(',')
            number_of_initials = len(all_text_after_first_comma_without_points) # число инициалов
            
            # Случай-исключение 2:
            fullname_list = [i.strip() for i in fullname_list]
            initials = fullname_list[1].split()
            number_of_ovna_evna_vich_vith_endings = [1 for i in initials if i[-4:] in patronymic_endings]
            number_of_ovna_evna_vich_vith_endings
            
            # Случай-исключение 1: если у нас белорусская фамилия – (brazevich, s. s. или gurevich, m. a.), то:
            # если после запятой у нас два инициала, 
            # И ХОТЯ БЫ ОДИН из инициалов меньше или два символов, то это скорее всего реальные настоящие И. О., 
            # поэтому эти случаи мы игнорируем
            # встречаемость: 35 случаев на 8537 изначальных строк; = 0.4%
            if number_of_initials == 2 and \
                len(all_text_after_first_comma_without_points[0]) <= 2 or\
                len(all_text_after_first_comma_without_points[1]) <= 2:
                    pass # это беларусы -- rutkevich, e. d.; игнорируем
            
            # Случай-исключение 2: белорусская фамилия с указанным отчеством – zubarevich, natalia vasilevna
            # если у того что за запятой на первом месте - natalia; или на втором - vasilevna;
            # окончание на evna, ovna, ovich, evich, то предположим что расстановка правильная - фамилия, и о
            # встречаемость: 3 случая на 8537 изначальных строк; = 0.035%
            elif number_of_ovna_evna_vich_vith_endings != [] and \
                     (first_part_of_name.endswith('vich') or first_part_of_name.endswith('vith')):
                    pass
            
            else:
                # ТУТ ЗАКОНЧИЛИСЬ белорусские ФАМИЛИИ!!!
                patronymic = fullname.split(', ')[0]
                last_name = fullname.split(', ')[1].split()[0]
                first_name = fullname.split(', ')[1].split()[1]
                correct_string = f'{last_name}, {first_name} {patronymic}' # пример: krasnokutskaya, olga alekseevna
                df.loc[index, 'author'] = correct_string
                counter += 1
    
print(f'Исправлено случаев: {counter}\n\n') # итоговая распространенность проблемы = 1.46%

last_names = get_unique_last_names(df, fullname_column='author', index_starts=1)
names_and_patronymics = get_unique_first_names_and_patronymics_combinations(df, fullname_column='author', index_starts=1)
print(f'После обработки обработки: {last_names.shape[0]} уникальных фамилий;')
print(f'После обработки: {names_and_patronymics.shape[0]} уникальных комбинаций имен-отчеств;')

До обработки: 5768 уникальных фамилий;
До обработки: 2311 уникальных комбинаций имен-отчеств;


Исправлено случаев: 125


После обработки обработки: 5748 уникальных фамилий;
После обработки: 2293 уникальных комбинаций имен-отчеств;


In [308]:
# 2.3 В некоторых случаях на первом месте находится не фамилия, а инициалы (и без точек, деления) (`213	"an, ershov"`) 
# -> проходимся циклом, делаем сравнения по длине части до запятой и части после запятой; 
# наверняка таких случаев будет не очень много и перестановку местами 
# (с последующим разбиением инициалов на две части) можно будет использовать как решение.
# ПРИМЕР: 
# an, ershov -----> ershov, an
# sd, lebedev ----> lebedev, sn
# НО: есть случаи когда у нас "два инициала", с ними мы тоже работаем (это юрьевичи; yu, barsukova svetlana)

counter = 0
print(f'До обработки: {last_names.shape[0]} уникальных фамилий;')
print(f'До обработки: {names_and_patronymics.shape[0]} уникальных комбинаций имен-отчеств;\n\n')

for index in range(1, df['author'].shape[0]+1):
    fullname = df.loc[index, 'author'] # тут у нас имя автора;
    if ',' in fullname:
        fullname_list = fullname.split(',') # тут у нас полный список: фамилия, имя отчество
        first_part_of_name = fullname_list[0]
        
        # Случай 1: у нас всего две составные части в слове пример an, ershov
        # встречаемость: 13 случаев на 8537 изначальных строк; = 0.15%
        if len(first_part_of_name) == 2 and len(fullname_list[1].split()) == 1:
            last_name = fullname.split(', ')[1]
            initials = fullname.split(', ')[0]
            correct_string = f'{last_name}, {initials}'
            df.loc[index, 'author'] = correct_string
            counter += 1
        
        # Случай 2: опять отчества! - и все Юрьевичи
        # встречаемость: 8 случаев на 8537 изначальных строк; = 0.09%
        # тут почему-то всегда проблема с yurievichivi.........
        # светлана барсукова - юрьевна;
        # андрей мягков – юрьевич;
        elif len(first_part_of_name) == 2 and len(fullname_list[1].split()) > 1 and fullname.startswith('yu,'):
            last_name = fullname.split()[-2].strip()
            if last_name.endswith('a'): # да, очень простое правило, но... барсуковА -> юрьевна; барсукоВ -> юрьевич
                patronymic = 'yurievna'
            else:
                patronymic = 'yurievich'
            first_name = fullname.split()[-1].strip('.')
            correct_string = f'{last_name}, {first_name}. {patronymic[:2]}.' # получается "barsukova, s. yu."
            df.loc[index, 'author'] = correct_string
            counter += 1
        else:
            pass

print(f'Было произведено {counter} замен/ы.\n\n')

last_names = get_unique_last_names(df, fullname_column='author', index_starts=1)
names_and_patronymics = get_unique_first_names_and_patronymics_combinations(df, fullname_column='author', index_starts=1)
print(f'После обработки обработки: {last_names.shape[0]} уникальных фамилий;')
print(f'После обработки: {names_and_patronymics.shape[0]} уникальных комбинаций имен-отчеств;')

До обработки: 5748 уникальных фамилий;
До обработки: 2293 уникальных комбинаций имен-отчеств;


Было произведено 21 замен/ы.


После обработки обработки: 5745 уникальных фамилий;
После обработки: 2278 уникальных комбинаций имен-отчеств;


In [309]:
# 2.4 В некоторых случаях на первом месте находится полное имя, а на втором - фамилия (`216	"anastasiya, varzakova"`) 
# -> нужно составить словарь с частотой встречаемости фамилий и имен; 
# и если то что на первом месте имеет большУю встречаемость в словаре с частотностью имен, 
# то поменять первое и второе местами.

# Часть 1: собираем датафрейм с именами!
# (тут происходило немного магии)
# я спарсила данные с сайта
# по мужским именам: https://ru.wiktionary.org/wiki/Индекс:Русский_язык/Мужские_имена
# и по женским именам: https://ru.wiktionary.org/wiki/Индекс:Русский_язык/Мужские_имена


# далее добавила имена которые можно вычленить из нашего names_and_patronymics pandas.DataFrame

#!pip install requests
#!pip install bs4
import requests
from bs4 import BeautifulSoup
from transliterate import translit

# 1.1. СОБИРАЕМ МУЖСКИЕ ИМЕНА
link = 'https://ru.wiktionary.org/wiki/Индекс:Русский_язык/Мужские_имена'
html = requests.get(link)
soup = BeautifulSoup(html.text)

male_names = [i.get('title') for i in soup.find_all('a') if i.get('title') != None]
male_names.pop(0)
end_index = male_names.index('Служебная:Категории')
male_names = male_names[:end_index]

# 1.2 СОБИРАЕМ ЖЕНСКИЕ ИМЕНА
link = 'https://ru.wiktionary.org/wiki/Индекс:Русский_язык/Женские_имена'
html = requests.get(link)
soup = BeautifulSoup(html.text)

female_names = [i.get('title').replace(" (страница не существует)", "") for i in soup.find_all('a') if i.get('title') != None]
female_names = [i for i in female_names if i.startswith('Редактировать') == False]
index_start = female_names.index('Категория:Женские имена') + 1
index_end = female_names.index('Служебная:Категории')
female_names = female_names[index_start:index_end]

# 2. ПОЛУЧАЕМ ДАТАФРЕЙМ
female_names_df = pd.DataFrame(female_names, columns=['name_ru'])
female_names_df['sex'] = 'female'
male_names_df = pd.DataFrame(male_names, columns=['name_ru'])
male_names_df['sex'] = 'male'
male_names_df['name_eng'] = male_names_df['name_ru'].apply(lambda x: translit(x, reversed=True))
female_names_df['name_eng'] = female_names_df['name_ru'].apply(lambda x: translit(x, reversed=True))
male_names_df['name_eng'] = male_names_df['name_eng'].apply(lambda x: x.replace("'", ""))
female_names_df['name_eng'] = female_names_df['name_eng'].apply(lambda x: x.replace("'", ""))
only_names_df = pd.concat([male_names_df, female_names_df])
only_names_df = only_names_df.drop(axis=1, columns=['sex'])

In [310]:
only_names_df # результат

,name_ru,name_eng
0,Аарон,Aaron
1,Август,Avgust
2,Авдей,Avdej
3,Аверьян,Averjan
4,Агафон,Agafon
...,...,...
624,Яна,Jana
625,Янина,Janina
626,Ярослава,Jaroslava
627,Ясна,Jasna


In [311]:
only_names = only_names_df['name_eng'].apply(lambda x: x.lower()).tolist()
only_names = [i for i in only_names if i not in ['herve', 'petrovic', 'jia', 'gella', 'lada', 'leon', 'ali', 'lukov', 'a']]
only_names.extend(['hector', 'amy']) # в эту строку можно добавить и другие имена и их разновидности латиницей (например, если вылезают ниже)
only_names[0:5] # тут у нас имена

['aaron', 'avgust', 'avdej', 'averjan', 'agafon']

In [312]:
# сюда же добавила то что было в names_dict - разные формы написания мужских и женских имен для большей корректности
print(len(only_names))
only_names.extend([i for i in names_dict.values()])
only_names.extend([i for i in names_dict.keys()])
only_names = set(only_names)
print(len(only_names))

900
1067


In [313]:
# Продолжение 2.4 В некоторых случаях на первом месте находится полное имя, а на втором - фамилия (`216	"anastasiya, varzakova"`) 
# -> нужно составить словарь с частотой встречаемости фамилий и имен; 
# и если то что на первом месте имеет большУю встречаемость в словаре с частотностью имен, 
# то поменять первое и второе местами.

# Часть 1: считаем самые частые фамилии; имена
counter = 0

print(f'До обработки: {last_names.shape[0]} уникальных фамилий;')
print(f'До обработки: {names_and_patronymics.shape[0]} уникальных комбинаций имен-отчеств;\n\n')
print('Примеры: ')

for index in range(1, df['author'].shape[0]+1):
    fullname = df.loc[index, 'author'] # тут у нас имя автора;
    if ',' in fullname:
        fullname_list = fullname.split(', ') # тут у нас полный список: фамилия, имя отчество
        first_part_of_name = fullname_list[0] # фамилия
        name_and_patronymic = fullname_list[1] # имя и потенциально отчество
        name = name_and_patronymic.split()[0] # как минимум первый инициал/имя автора
        
        # Если то что на первом месте ("фамилия") находится в списке с именами и следующее далее имя не находится
        # в списке с именами (исключение: "artem, olga"), то:
        if first_part_of_name in only_names and name not in only_names:
            # ИГНОРИРИУЕМ Сase 1: на самом деле является фамилией – "aleksander, j."; "ivan, e."; "bogdan, i. v."
            if name_and_patronymic.strip().count('.') == 2 or len(name_and_patronymic) < 3:
                pass
            
            # РАБОТАЕМ Case 2: два элемента в строчке "alexander, kurakin"
            elif len(fullname.replace(',', '').split()) == 2 or fullname.endswith('.'):
                real_name = first_part_of_name
                real_last_name = name
                correct_string = f'{real_last_name}, {real_name}' # "kurakin, alexander"
                df.loc[index, 'author'] = correct_string # замена
                # демонстрация
                print(fullname)
                print(correct_string, end='\n\n')
                counter += 1
                
            # РАБОТАЕМ Case 3: три элемента в строчке; "katerina, sergeevna kryuchkova" 
            else:
                # Case 3.1: "katerina, sergeevna kryuchkova" 
                if fullname.split(', ')[1].split()[0][-4:] in patronymic_endings:
                    real_patronymic = name_and_patronymic.split()[0]
                    real_name = first_part_of_name
                    real_last_name = name_and_patronymic.split()[1]
                # Case 3.2: "irina, popova petrovna" 
                else:
                    real_patronymic = name_and_patronymic.split()[1]
                    real_name = first_part_of_name
                    real_last_name = name_and_patronymic.split()[0]
                correct_string = f'{real_last_name}, {real_name} {real_patronymic}'
                df.loc[index, 'author'] = correct_string # замена
                # демонстрация
                print(fullname)
                print(correct_string, end='\n\n')
                counter += 1
            
print(f'Исправлено случаев: {counter}', end='\n\n')

last_names = get_unique_last_names(df, fullname_column='author', index_starts=1)
names_and_patronymics = get_unique_first_names_and_patronymics_combinations(df, fullname_column='author', index_starts=1)
print(f'После обработки обработки: {last_names.shape[0]} уникальных фамилий;')
print(f'После обработки: {names_and_patronymics.shape[0]} уникальных комбинаций имен-отчеств;')

До обработки: 5745 уникальных фамилий;
До обработки: 2278 уникальных комбинаций имен-отчеств;


Примеры: 
alexander, kurakin
kurakin, alexander

alexandra, dmitrieva
dmitrieva, alexandra

anastasiya, varzakova
varzakova, anastasiya

artem, samuilovith kyzlasov
kyzlasov, artem samuilovith

bella, valeryevna kaziyeva
kaziyeva, bella valeryevna

bogdan, gnat v.
gnat, bogdan

elena, sergeevna ionkina
ionkina, elena sergeevna

elena, vishlenkova
vishlenkova, elena

elena, vladimirovna tyumentseva
tyumentseva, elena vladimirovna

irina, grigoryeva
grigoryeva, irina

irina, mitrofanova
mitrofanova, irina

irina, pervova
pervova, irina

irina, yuryevna gedgafova
gedgafova, irina yuryevna

iuliia, toktamysheva
toktamysheva, iuliia

katerina, sergeevna kryuchkova
kryuchkova, katerina sergeevna

liana, munirovna rakhimova
rakhimova, liana munirovna

lyudmila, alexandrova
alexandrova, lyudmila

maria, volynskaia
volynskaia, maria

nadezhda, radina
radina, nadezhda

natalia, komarova
komarova, nata

In [314]:
# 2.5 Приведение записей к единой форме `фамилия, и. о.`. 
# На предыдущих этапах мы поработали с корректным порядком записи имен, фамилий и отчеств, 
# и как мне кажется, теперь достаточно спокойно можем сократить изначальные строки до корректных записей.

for index in range(1, df['author'].shape[0]+1):
    fullname = df.loc[index, 'author'] # тут у нас имя автора;
    # если у нас есть запятая в имени автора, то
    if ',' in fullname:
        fullname_list = fullname.split(', ') # тут у нас полный список: фамилия, имя отчество
        fullname_list = [i.strip() for i in fullname_list]
        last_name = fullname_list[0] # фамилия
        first_name_and_patronymic = "".join(fullname_list[1:]) # имя, и, если есть, отчество
        #print(last_name, first_name_and_patronymic)
        # Case 1: Есть и имя, и отчество,
        # bovaeva, galina mandzhievna  ->  bovaeva, g. m
        if len(first_name_and_patronymic.split()) == 2:
            name = first_name_and_patronymic.split()[0]
            patronymic = first_name_and_patronymic.split()[1]
            correct_string = f'{last_name}, {name[0]}. {patronymic[0]}.'
        # Case 2: Есть только имя:
        # boyer, pascal  ->  boyer, p.
        elif len(first_name_and_patronymic.split()) == 1:
            # Case 2.1 Есть только имя:
            if len(first_name_and_patronymic) != 2:
                name = first_name_and_patronymic.split()[0]
                correct_string = f'{last_name}, {name[0]}.'
            # Case 2.2 Есть имя и инициалы: zyuganov, ga -> zyuganov, g. a
            else:
                name = first_name_and_patronymic[0]
                patronymic = first_name_and_patronymic[1]
                correct_string = f'{last_name}, {name[0]}. {patronymic[0]}.'
        # Case 3: иные случаи, когда много знаков после инициалов
        # aganov, andrey g. v.  ->  aganov, a. g. v
        else:
            correct_string = f'{last_name}, {". ".join([i[0] for i in first_name_and_patronymic.split()])}'
        # Тут воспроизводится замена
        df.loc[index, 'author'] = correct_string 

In [315]:
df['author_old_name'] = old_author_names

In [316]:
df # теперь в author_old_name - изначальное название автора

,author,documents,citations,total link strength,author_old_name
1,"abaeva, a. m.",1,1,4,"abaeva, a. m."
2,"abaeva, a. m.",1,0,4,"abaeva, asiyat m."
3,"abaeva, o. p.",2,0,8,"abaeva, o. p."
4,"abalakin, a. a.",1,0,3,"abalakin, a. a."
5,"abalakina, t. v.",1,0,3,"abalakina, t. v."
...,...,...,...,...,...
8533,"zyrianov, v. v.",1,0,3,"zyrianov, v. v."
8534,"zyryanov, v. v.",2,3,3,"zyryanov, v. v."
8535,"zyryanov, v. v.",1,0,2,"zyryanov, vladimir v."
8536,"zyuganov, g. a.",1,0,20,"zyuganov, ga"


In [317]:
n_stage_1 = df['author'].unique().shape[0] # сколько авторов уникальных после первого этапа

In [318]:
last_names = get_unique_last_names(df, fullname_column='author', index_starts=1)
names_and_patronymics = get_unique_first_names_and_patronymics_combinations(df, fullname_column='author', index_starts=1)
print(f'После обработки обработки: {last_names.shape[0]} уникальных фамилий;')
print(f'После обработки: {names_and_patronymics.shape[0]} уникальных комбинаций имен-отчеств;')

После обработки обработки: 5745 уникальных фамилий;
После обработки: 509 уникальных комбинаций имен-отчеств;


In [319]:
df['author'].unique().shape[0] # на самом деле, у нас 7579 уникальных авторов в данных

7579

In [320]:
print(f'Изначально было {df.shape[0]} уникальных авторов;')
print(f'Только методам выше было сокращено {df.shape[0] - df["author"].unique().shape[0]} имен авторов')
print(f'То есть, {(df.shape[0] - df["author"].unique().shape[0]) / df.shape[0] * 100}% записей')

Изначально было 8537 уникальных авторов;
Только методам выше было сокращено 958 имен авторов
То есть, 11.221740658310882% записей


In [321]:
df.to_excel('preliminary_data.xlsx') # предварительные данные для сверки

## Пункт 3. Исправление имен авторов ##

**Что будет исправлено?**

Мы рассчитываем расстояние Левенштейна для всех пар имен, и при совпадении инициалов и отличии фамилии на 1 сиимвол (1 по метрике расстояния Левенштейна) мы сохраняем пары этих элементов для дальнейшего преобразования к наиболее полной форме.

```
aleshkovski, i. a.
aleshkovski, i.

alexandrov, d. a.
alexandrov, d.

allahverdyan, a. g.
allakhverdyan, a.

```

**По каким правилам?**
Мне кажется, что максимально снизить число уникальных авторов - хорошая цель, но нам также важно не потерять слишком много авторов, иначе данные в ходе анализа получатся некорректные. Поэтому я не просто производила замены согласно синтаксической близости фамилий авторов, а еще основываясь на дополнительных правилах чтобы иметь высокую уверенность в корректности наших данных:
1. Фамилиии двух авторов, которые сравниваются, обе не меньше 5 символов;
2. У двух фамилий одинаковое окончание (чтобы далее мы случайно не заменили мужские фамилии женскими и женские - мужскими);
3. У двух фамилий одинаковая первая буква;
4. При этом у нас либо одинаковые инициалы, либо совпадает первая буква (то есть, имя), пример: `aleshkovski, i. a.`, `aleshkovski, i.`
5. И расстояние Левенштейна равно единице.

В случае выполнения всех этих пяти условий мы можем подобрать фамилии для взаимозамены.

**Что еще можно сделать?**
1. Потенциально исследовать случаи, когда возможно инициалы перепутаны местами?

```
aleynikov, a. v.
aleynikov, v. a.
```

2. Дополнить код, добавив проверку на отличающийся символ – является ли это гласной/согласной, чтобы исключить некорректные замены (реализовано в коде ниже):
```
goryunov, a. v.
gorbunov, a.
```
    2.1. Поработать с группой фамилиий, где отличия в согласном звуке (добавить проверку на звуки?) - потому что в основном фамилии отличаются на конкретные звуки либо отличаются на другие, и замена будет уже некорректной. 
```
'slisovskiy, d. e.': ['slizovskiy, d. e.'], # Звуки s-z
 'strisoe, a. l.': ['strizoe, a. l.'], # Звуки s-z
 'sukhorukov, v. v.': ['suhorukov, v. v.'], # Звуки kh-h
 'terehov, a. i.': ['terekhov, a. i.'],  # Звуки kh-h
 'toschenko, z. t.': ['toshchenko, z. t.', 'toshchenko, z.'],  # Звуки sch-shch (лишняя h) в звуке щ
 'tsvetkov, o. m.': ['tzvetkov, o.'],  # Звуки s-z
 'ushakin, s. a.': ['uschakin, s.'],  # Звуки sh-sc
 'valiahmetov, r. m.': ['valiakhmetov, r. m.'],  # Звуки kh-h
 'verhovin, v. i.': ['verkhovin, v. i.'],  # Звуки kh-h
 'volchkova, l. t.': ['voltchkova, l.'],  # Звуки ch-tch (лишняя t)
```
    

3. Исключить из рассмотрения распространенные фамилии "Иванов", "Петров", "Кузнецов", "Попов", "Васильев" и. т. д.  или сравнивать их более строго?

In [322]:
# работа с расстоянием левенштейна
from Levenshtein import distance as lev
import re
 
def is_vowel(char):
    ''' Возвращает True если знак на английском - гласный; False - если согласный*.
    __________________________________________
    * Примечание: знак y,Y мы считаем гласным.
    __________________________________________
    Параметры:
            :char(string) - символ на английском;
    __________________________________________
    Пример:
        >result = is_vowel('a')
        >print(result) # True
    '''
    if re.match(r'[aeiouyjAEIOUYJ]', char):
        return True
    return False

def is_two_characters_one_type(a_char, b_char):
    '''Возвращает True если два знака одного типа - гласные (vowel) или согласные (consonant), и False в ином случае.
    __________________________________________
    Параметры:
            :a_char(str) - строка на английском;
            :b_char(str) - строка на английском;
    __________________________________________
    Пример:
        >result = is_two_characters_one_type("y", "i")
        >print(result) # True
    '''
    a_type = is_vowel(a_char)
    b_type = is_vowel(b_char)
    if sum([a_type, b_type]) in [0, 2]:
        return True
    return False

def get_first_different_character_for_a(a_string, b_string):
    '''Возвращает первый отличный знак для строки a по отношению к строке b.
    __________________________________________
    Параметры:
            :a_string(string) - первая строка;
            :b_string(string) - вторая строка.
    __________________________________________
    Пример:
        >result_a = get_first_different_character_for_a('zhuravskaia, t. n.', 'zhuravskaya, t. n.')
        >print(result_a) # i
    '''
    for index, char in enumerate(a_string):
        if not b_string[index] == char:
            return char

def get_first_different_character_for_b(a_string, b_string):
    '''Возвращает первый отличный знак для строки b по отношению к строке a.
    __________________________________________
    Параметры:
            :a_string(string) - первая строка;
            :b_string(string) - вторая строка.
    __________________________________________
    Пример:
        >result_b = get_first_different_character_for_b('zhuravskaia, t. n.', 'zhuravskaya, t. n.')
        >print(result_b) # y
    '''
    for index, char in enumerate(b_string):
        if not a_string[index] == char:
            return char

a_string = 'kultigin, v.'
b_string = 'kultugin, v.'
print(get_first_different_character_for_a(a_string, b_string)) # i
print(get_first_different_character_for_b(a_string, b_string)) # u

i
u


In [323]:
# как работает алгоритм?
author_1 = 'kultigin, v.'
author_2 = 'kultugin, v.'
first_difference_author_1 = get_first_different_character_for_a(author_1, author_2) # i
first_difference_author_2 = get_first_different_character_for_b(author_1, author_2) # u
result_of_comparison = is_two_characters_one_type(first_difference_author_1, first_difference_author_2)
type_1 = is_vowel(first_difference_author_1)
type_2 = is_vowel(first_difference_author_2)

if result_of_comparison == True and type_1 == True:
    print('Все отличающиеся субстроки - vowel - гласные')
elif result_of_comparison == True and type_1 == False:
    print('Все отличающиеся субстроки - consonant - согласные')
else:
    print('Одна субстрока - vowel, другая consonant')

Все отличающиеся субстроки - vowel - гласные


In [324]:
# Есть исключение:
author_1 = 'golosenko, i.'
author_2 = 'golossenko, i.'
first_difference_author_1 = get_first_different_character_for_a(author_1, author_2) # e
first_difference_author_2 = get_first_different_character_for_b(author_1, author_2) # s

result_of_comparison = is_two_characters_one_type(first_difference_author_1, first_difference_author_2)
type_1 = is_vowel(first_difference_author_1) # vowel (e)
type_2 = is_vowel(first_difference_author_2) # consonant (s)
print(type_1, type_2)
# Берем слово, в котором первый отличающийся звук - согласный
if type_2 == False:
    index_of_first_dif_litera = author_2.index(first_difference_author_2) # берем индекс первого отличающегося звука
    index_of_next_litera = index_of_first_dif_litera + 1
    if author_2[index_of_first_dif_litera] == author_2[index_of_next_litera]:
        print(f'Подстрока {author_2[index_of_next_litera]} в "{author_2}" дублируется')
    else:
        pass
elif type_1 == False:
    index_of_first_dif_litera = author_1.index(first_difference_author_1) # берем индекс первого отличающегося звука
    index_of_next_litera = index_of_first_dif_litera + 1
    if author_1[index_of_first_dif_litera] == author_1[index_of_next_litera]:
        print(f'Подстрока {author_1[index_of_next_litera]} в "{author_1}" дублируется')
    else:
        pass

True False
Подстрока s в "golossenko, i." дублируется


In [325]:
# код для поиска совпадений

group_1_dict = {} # гласные - vowel
group_2_dict = {} # согласные - consonant
group_3_dict = {} # дублирующие звуки
group_4_dict = {} # разные звуки - гласные/согласные

author_1_list = [i for i in df['author'].tolist() if ', ' in i]
author_2_list = [i for i in df['author'].tolist() if ', ' in i]

# идем по авторам
for author_1 in tqdm.tqdm(author_1_list):
    # если у нас длина фамилии не меньше 4
    if len(author_1.split(', ')[0]) > 4:
        # данные автора 1
        last_name_1 = author_1.split(', ')[0]
        initials_1 = author_1.split(', ')[1].split()
        #print(last_name_1)
        for author_2 in author_2_list:
            ## данные автора_2
            last_name_2 = author_2.split(', ')[0]
            initials_2 = author_2.split(', ')[1].split()
            # Если у нас длина фамилии не меньше 4 И ПРИ ЭТОМ окончания у фамилий одинаковые
            # И при этом первая буква тоже одинаковая (иначе - )
            # и при этом это не одни и те же строки (в точности; author_1 != author_2)
            # Почему важно? Род; чтобы мы случайно не заметчили "abalakina" и "abalakin"
            # И расстояние левенштейна == 1
            if len(author_2.split(', ')[0]) > 4 and\
                last_name_1[-1] == last_name_2[-1] and\
                ((last_name_1[0] == last_name_2[0]) | ((last_name_1.startswith('ch') and last_name_2.startswith('tch')) or (last_name_1.startswith('tch') and last_name_2.startswith('ch'))) | (last_name_1[0] in ['k', 'c'] and last_name_2[0] in ['k', 'c'])) and\
                author_1 != author_2 and\
                lev(last_name_1, last_name_2) == 1:
                # И расстояние левенштейна 1
                # И у нас одинаковые инициалы или совпадает первый инициал ->
                # записываем в словарь
                # этот способ позволяет не мэтчить авторов вслепую, а уделить больше внимания инициалам, не оставлять их без внимания

                lev_distance = lev(last_name_1, last_name_2)
                flag_1 = len(initials_1) == 2 # в имени первого автора два инициала? True/False
                flag_2 = len(initials_2) == 2 # в имени второго автора два инициала? True/False
                flag_3 = False # совпадают ли инициалы в достаточной мере?

                ## Case 1: два инициала у каждого имени (то есть, и имя, и отчество) или только один -> да, меняем
                if sum([flag_1, flag_2]) in [0, 2]:
                    if initials_1 == initials_2:
                        flag_3 = True 
        
                ## Case 2: (два инициала у первого и один у второго) ИЛИ (один инициал у второго)
                else:
                    ### пример: aleksandrov, m. a. И aleksandrov, m.
                    if initials_1[0] == initials_2[0] or sorted([initials_1[0], initials_2[0]]) == ['h', 'k']:
                        flag_3 = True
                        
                # Если все ок, обновляем словарь
                if flag_3 == True:
                    if (flag_1 < flag_2):
                        value_for_key = author_2
                        value_for_value = author_1
                    else:
                        value_for_key = author_1
                        value_for_value = author_2
                        
                    # Теперь - проверяем на согласные
                    first_difference_author_1 = get_first_different_character_for_a(author_1, author_2) # i
                    first_difference_author_2 = get_first_different_character_for_b(author_1, author_2) # u
                    result_of_comparison = is_two_characters_one_type(first_difference_author_1, first_difference_author_2)
                    type_1 = is_vowel(first_difference_author_1)
                    type_2 = is_vowel(first_difference_author_2)
                    
                    # Case 1: отличающиеся символы - гласные (высокая уверенность);
                    if result_of_comparison == True and type_1 == True:
                        if value_for_key not in group_1_dict:
                            group_1_dict[value_for_key] = [value_for_value]
                        else:
                            group_1_dict[value_for_key].append(value_for_value)
                    # Case 2: отличающиеися символы - согласные (средняя уверенность)
                    elif result_of_comparison == True and type_1 == False:
                        if value_for_key not in group_2_dict:
                            group_2_dict[value_for_key] = [value_for_value]
                        else:
                            group_2_dict[value_for_key].append(value_for_value)
                    # Case 3: отличающиеся символы различаются по типу, НО 'golosenko, i.', 'golossenko, i.',
                    # Case 4: отличающиеся символы отличаются по типу (гласная и согласная)
                    else:
                        # проверяем символы на дублирование
                        if type_2 == False:
                            index_of_first_dif_litera = author_2.index(first_difference_author_2) # берем индекс первого отличающегося звука
                            index_of_next_litera = index_of_first_dif_litera + 1
                            # Подстрока author_2[index_of_next_litera] в author_2 дублируется -> group_3_dict
                            if author_2[index_of_first_dif_litera] == author_2[index_of_next_litera]:
                                if value_for_key not in group_3_dict:
                                    group_3_dict[value_for_key] = [value_for_value]
                                else:
                                    group_3_dict[value_for_key].append(value_for_value)
                            # Подстрока не дублируется -> group_4_dict
                            else:
                                if value_for_key not in group_4_dict:
                                    group_4_dict[value_for_key] = [value_for_value]
                                else:
                                    group_4_dict[value_for_key].append(value_for_value)
                        elif type_1 == False:
                            index_of_first_dif_litera = author_1.index(first_difference_author_1) # берем индекс первого отличающегося звука
                            index_of_next_litera = index_of_first_dif_litera + 1
                            # Подстрока author_1[index_of_next_litera] в author_1 дублируется.
                            if author_1[index_of_first_dif_litera] == author_1[index_of_next_litera]:
                                if value_for_key not in group_3_dict:
                                    group_3_dict[value_for_key] = [value_for_value]
                                else:
                                    group_3_dict[value_for_key].append(value_for_value)
                            # Подстрока не дублируется -> group_4_dict
                            else:
                                if value_for_key not in group_4_dict:
                                    group_4_dict[value_for_key] = [value_for_value]
                                else:
                                    group_4_dict[value_for_key].append(value_for_value)

                    try:
                        author_1_list.remove(author_2)
                    except:
                        pass
                    try:
                        author_2_list.remove(author_1)
                    except:
                        pass
                    try:
                        author_2_list.remove(author_2)
                    except:
                        pass

 94%|███████████████████████████████████▊  | 8042/8523 [01:20<00:04, 100.39it/s]


In [326]:
group_1_dict = {k:list(set(v)) for k,v in group_1_dict.items()} # группа 1 - различия в гласных
group_1_dict

{'afanasev, v. v.': ['afanasiev, v. v.'],
 'aimautova, n. e.': ['aymautova, n. e.'],
 'akaev, v.': ['akayev, v.'],
 'aleinikov, a. v.': ['aleynikov, a. v.'],
 'dmitriyeva, a. v.': ['dmitrieva, a.'],
 'andreenkov, v. g.': ['andrienkov, v. g.'],
 'anourin, v. f.': ['anurin, v. f.'],
 'archangelskiy, v. n.': ['archangelsky, v. n.'],
 'ardalianova, a. y.': ['ardalyanova, a. y.'],
 'artiushina, a. v.': ['artyushina, a. v.'],
 'arutunian, y. v.': ['arutiunian, y.'],
 'arutiunova, e. m.': ['arutyunova, e. m.'],
 'avksentev, v. a.': ['avksentiev, v. a.', 'avksentyev, v. a.'],
 'azieva, r. k.': ['aziyeva, r. k.'],
 'azieva, r. h.': ['aziyeva, r. h.'],
 'bagdasarian, n. g.': ['bagdasaryan, n. g.'],
 'barsoukova, s. y.': ['barsukova, s. y.'],
 'baykov, a. a.': ['bykov, a.'],
 'bedniy, b. i.': ['bedny, b. i.'],
 'bykova, s. n.': ['bekova, s.'],
 'belenkiy, v. k.': ['belenky, v. k.'],
 'beliaev, a. m.': ['belyaev, a.'],
 'beliaeva, l. a.': ['beliayeva, l. a.', 'belyaeva, l. a.'],
 'bessokirnaia, g.

In [327]:
group_2_dict = {k:list(set(v)) for k,v in group_2_dict.items()} # группа 2 - различия в согласных
group_2_dict # с этой группо можно конкретно поработать

{'allahverdyan, a. g.': ['allakhverdyan, a. g.'],
 'archangelskiy, v. n.': ['arkhangelskiy, v. n.'],
 'arefiev, a. l.': ['areviev, a. l.'],
 'astvatsaturova, m. a.': ['asvatsaturova, m. a.'],
 'bekov, m. b.': ['belov, m.'],
 'belova, s. u.': ['bekova, s.'],
 'belova, s. v.': ['bekova, s.'],
 'biryukov, p. b.': ['bizyukov, p.'],
 'bokov, v. i.': ['bobkov, v.'],
 'bocharov, v. y.': ['botcharov, v. y.'],
 'borsch, i.': ['borshch, i.'],
 'braiovich, a. s.': ['braiovitch, a. s.'],
 'brazevich, s. s.': ['brazevitch, s. s.'],
 'bultseva, m. a.': ['burtseva, m. a.'],
 'burhanova, f. b.': ['burkhanova, f. b.'],
 'bysov, l. g.': ['byzov, l. g.'],
 'cansuzian, l. v.': ['kansuzian, l. v.'],
 'cherednichenko, g. a.': ['cherednitchenko, g. a.'],
 'chernih, a. i.': ['chernikh, a. i.'],
 'chernykh, a. b.': ['chernysh, a.'],
 'chirikova, a. e.': ['tchirikova, a. e.'],
 'chudnov, i. a.': ['tchudnov, i. a.'],
 'konroy, n. v.': ['conroy, n.'],
 'corel, l. v.': ['korel, l. v.'],
 'driahlov, n. i.': ['driak

In [328]:
group_3_dict = {k:list(set(v)) for k,v in group_3_dict.items()} # группа 3  - различия в дублирующихся звуках
group_3_dict 

{'abasov, z. a.': ['abassov, z. a.'],
 'denisenko, m. b.': ['denissenko, m.'],
 'denisov, b. p.': ['denissov, b. p.'],
 'efendiev, a. g.': ['effendiev, a. g.'],
 'golosenko, i. a.': ['golossenko, i. a.'],
 'kiselev, i. y.': ['kisselev, i. y.'],
 'kosova, l. b.': ['kossova, l. b.'],
 'krasin, y. a.': ['krassin, y. a.'],
 'lokosov, v. v.': ['lokossov, v. v.'],
 'lysova, a. v.': ['lyssova, a. v.'],
 'moiseeva, n. a.': ['moisseeva, n. a.'],
 'osadchaya, g. i.': ['ossadchaya, g. i.'],
 'osipov, g. v.': ['ossipov, g. v.'],
 'osipov, p. n.': ['ossipov, p. n.'],
 'tadevosian, e. v.': ['tadevossian, e. v.'],
 'vasilev, v. p.': ['vassilev, v. p.'],
 'vasiliev, i. a.': ['vassiliev, i. a.'],
 'veselkova, n. v.': ['vesselkova, n. v.'],
 'veselov, y. v.': ['vesselov, y. v.'],
 'vlasov, v. v.': ['vlassov, v. v.']}

In [329]:
# Группа 4: гласные и согласные различаются; 
# Однако, все еще есть хорошие потенциальные замены:
# 'cherdnichenko, g. a.': ['cherednichenko, g. a.'],
# 'zintsova, a.': ['zintsoya, a.']
group_4_dict = {k:list(set(v)) for k,v in group_4_dict.items()} # группа 4 - различия в гласных-согласных
group_4_dict 

{'altukhov, s.': ['altukov, s.'],
 'avraamova, e. m.': ['avramova, e. m.'],
 'baimbetov, a. a.': ['bambetov, a. a.'],
 'bazanov, v. a.': ['bazhanov, v. a.'],
 'belenkiy, v. k.': ['belenkly, v. k.'],
 'bobkov, v. n.': ['boikov, v.'],
 'boikov, v. e.': ['bobkov, v.'],
 'borisov, v. a.': ['bortsov, v. a.'],
 'bourganov, a. k.': ['bourghanov, a. k.'],
 'budanova, l. y.': ['buyanova, l. y.'],
 'cherdnichenko, g. a.': ['cherednichenko, g. a.'],
 'danovski, s. l.': ['danovskii, s. l.'],
 'dorogkin, j. n.': ['dorojkin, j. n.'],
 'drobiizheva, l. m.': ['drobizheva, l. m.'],
 'dvoimenniy, i. a.': ['dvoimennty, i. a.'],
 'dzhutsev, k. v.': ['dzutsev, k. v.'],
 'filonovitch, s. r.': ['filontovitch, s. r.'],
 'gendin, a. m.': ['ghendin, a. m.'],
 'gertchikov, v. i.': ['ghertchikov, v. i.'],
 'ghilmanov, a. z.': ['gilmanov, a. z.'],
 'gorbunov, a. v.': ['goryunov, a. v.'],
 'igbitkhanian, e. d.': ['igitkhanian, e. d.'],
 'ighitkhanian, e. d.': ['igitkhanian, e. d.'],
 'kraeva, o. l.': ['karaeva, o.'

К сожалению, помимо реально стоящих замен попадаются и проблемные замены; в большинстве своем они попадают в четвертую группу (в двух строках различается гласная и согласная):
```
'bykova, s.': ['bekova, s.'], # 1 группа
'belova, s. v.': ['bekova, s.'], # 2 группа
'gainer, m.': ['giner, m.'], # 2 группа
'goryunov, a. v.': ['gorbunov, a.'], # 4 группа
'ivanova, a. e.': ['ivankova, a.'] # 4 группа
'kasavin, i. t.': ['krasavin, i.'], # 4 группа
'kuzin, i.': ['kuzmin, i.'], # 4 группа
```

Но для всех остальных историй - 1 группы (различия в двух гласных звуках), 2 группы (различиях в двух согласных звуках), 3 группы (дублирующиеся звуки) – замены должны достаточно хорошо работать

In [330]:
df['n_of_initials'] = df['author'].apply(lambda x: x.count('.'))

In [331]:
df

,author,documents,citations,total link strength,author_old_name,n_of_initials
1,"abaeva, a. m.",1,1,4,"abaeva, a. m.",2
2,"abaeva, a. m.",1,0,4,"abaeva, asiyat m.",2
3,"abaeva, o. p.",2,0,8,"abaeva, o. p.",2
4,"abalakin, a. a.",1,0,3,"abalakin, a. a.",2
5,"abalakina, t. v.",1,0,3,"abalakina, t. v.",2
...,...,...,...,...,...,...
8533,"zyrianov, v. v.",1,0,3,"zyrianov, v. v.",2
8534,"zyryanov, v. v.",2,3,3,"zyryanov, v. v.",2
8535,"zyryanov, v. v.",1,0,2,"zyryanov, vladimir v.",2
8536,"zyuganov, g. a.",1,0,20,"zyuganov, ga",2


In [332]:
def change_author_names_with_dictionary(df, dict_for_change, column_name='author'):
    '''Возвращает датафрейм с обновленными именами авторов в соответствие со словарём dict_for_change.
    __________________________________________
    Параметры:
            :df(pandas.DataFrame) - датафрейм с именами авторов;
            :dict_for_change(dict) - словарь, где ключ - это имя автора НА которое меняем, значения - имена, которые заменяются;
            :column_name = 'author' - название колонки с именами авторов.
    __________________________________________
    Пример:
        >new_df = change_author_names_with_dictionary(df, dict_for_change=group_1_dict, column_name='author')
        >new_df
    '''
    # Этап 0. Инициирование переменных
    new_df = df.copy()
    counter = 0
    
    # Этап 1. Прохождение по словарям
    for k,v in dict_for_change.items():
        one_author_list_of_names = v
        one_author_list_of_names.append(k)
        one_author_df = df[df[column_name].isin(one_author_list_of_names)]
        one_author_df = one_author_df.sort_values(by=['n_of_initials', 'documents'], ascending=False)
        index_of_first_author = one_author_df.index[0] # индекс автора, полное имя которого берем за основу; 
        needed_author_name = df.loc[index_of_first_author, column_name] # имя автора НА которое будем менять
        index_for_one_author = one_author_df.index # array с индексами; для этих индексов меняем имена авторов
        new_df.loc[index_for_one_author, column_name] = needed_author_name # замена
        counter += one_author_df.shape[0] - 1
    
    print(f'Было произведено {counter} замен.')
    return new_df

In [333]:
print(f'До обработки: {last_names.shape[0]} уникальных фамилий;')
print(f'До обработки: {names_and_patronymics.shape[0]} уникальных комбинаций имен-отчеств;\n\n')

df = change_author_names_with_dictionary(df, dict_for_change=group_1_dict, column_name='author')
df = change_author_names_with_dictionary(df, dict_for_change=group_2_dict, column_name='author')
df = change_author_names_with_dictionary(df, dict_for_change=group_3_dict, column_name='author')

last_names = get_unique_last_names(df, fullname_column='author', index_starts=1)
names_and_patronymics = get_unique_first_names_and_patronymics_combinations(df, fullname_column='author', index_starts=1)
print()
print(f'После обработки обработки: {last_names.shape[0]} уникальных фамилий;')
print(f'После обработки: {names_and_patronymics.shape[0]} уникальных комбинаций имен-отчеств;')

До обработки: 5745 уникальных фамилий;
До обработки: 509 уникальных комбинаций имен-отчеств;


Было произведено 380 замен.
Было произведено 167 замен.
Было произведено 35 замен.
После обработки обработки: 5488 уникальных фамилий;
После обработки: 508 уникальных комбинаций имен-отчеств;


In [334]:
# # Идем по авторам, ключам-значениям ПЕРВОЙ ГРУППЫ: различия в гласных; примеры:
# # arutiunian, y., arutiunian, y., arutunian, y. v.
# # Какой вариант произношения выбираем? Тот, где есть имя и отчетство, 
# counter = 0
# for k,v in group_1_dict.items():
#     one_author_list_of_names = v
#     one_author_list_of_names.append(k)
#     one_author_df = df[df['author'].isin(one_author_list_of_names)]
#     one_author_df = one_author_df.sort_values(by=['n_of_initials', 'documents'], ascending=False)
#     index_of_first_author = one_author_df.index[0] # индекс автора, полное имя которого берем за основу; 
#     needed_author_name = df.loc[index_of_first_author, 'author']
#     index_for_one_author = one_author_df.index # array с индексами; для этих индексов меняем имена авторов
# #     print(one_author_df)
# #     print(index_of_first_author, needed_author_name)
# #     print(index_for_replace)
#     df.loc[index_for_one_author, 'author'] = needed_author_name
#     counter += one_author_df.shape[0] - 1

# print(f'Было произведено {counter} замен.')

In [336]:
# код для поиска совпадений

group_1_dict = {} # гласные - vowel
group_2_dict = {} # согласные - consonant
group_3_dict = {} # дублирующие звуки
group_4_dict = {} # разные звуки - гласные/согласные

author_1_list = [i for i in df['author'].tolist() if ', ' in i]
author_2_list = [i for i in df['author'].tolist() if ', ' in i]

# идем по авторам
for author_1 in tqdm.tqdm(author_1_list):
    # если у нас длина фамилии не меньше 4
    if len(author_1.split(', ')[0]) > 4:
        # данные автора 1
        last_name_1 = author_1.split(', ')[0]
        initials_1 = author_1.split(', ')[1].split()
        #print(last_name_1)
        for author_2 in author_2_list:
            ## данные автора_2
            last_name_2 = author_2.split(', ')[0]
            initials_2 = author_2.split(', ')[1].split()
            # Если у нас длина фамилии не меньше 4 И ПРИ ЭТОМ окончания у фамилий одинаковые
            # И при этом первая буква тоже одинаковая (иначе - )
            # и при этом это не одни и те же строки (в точности; author_1 != author_2)
            # Почему важно? Род; чтобы мы случайно не заметчили "abalakina" и "abalakin"
            # И расстояние левенштейна == 1
            if len(author_2.split(', ')[0]) > 4 and\
                last_name_1[-1] == last_name_2[-1] and\
                ((last_name_1[0] == last_name_2[0]) | ((last_name_1.startswith('ch') and last_name_2.startswith('tch')) or (last_name_1.startswith('tch') and last_name_2.startswith('ch'))) | (last_name_1[0] in ['k', 'c'] and last_name_2[0] in ['k', 'c'])) and\
                author_1 != author_2 and\
                lev(last_name_1, last_name_2) == 1:
                # И расстояние левенштейна 1
                # И у нас одинаковые инициалы или совпадает первый инициал ->
                # записываем в словарь
                # этот способ позволяет не мэтчить авторов вслепую, а уделить больше внимания инициалам, не оставлять их без внимания

                lev_distance = lev(last_name_1, last_name_2)
                flag_1 = len(initials_1) == 2 # в имени первого автора два инициала? True/False
                flag_2 = len(initials_2) == 2 # в имени второго автора два инициала? True/False
                flag_3 = False # совпадают ли инициалы в достаточной мере?

                ## Case 1: два инициала у каждого имени (то есть, и имя, и отчество) или только один -> да, меняем
                if sum([flag_1, flag_2]) in [0, 2]:
                    if initials_1 == initials_2:
                        flag_3 = True 
        
                ## Case 2: (два инициала у первого и один у второго) ИЛИ (один инициал у второго)
                else:
                    ### пример: aleksandrov, m. a. И aleksandrov, m.
                    if initials_1[0] == initials_2[0] or sorted([initials_1[0], initials_2[0]]) == ['h', 'k']:
                        flag_3 = True
                        
                # Если все ок, обновляем словарь
                if flag_3 == True:
                    if (flag_1 < flag_2):
                        value_for_key = author_2
                        value_for_value = author_1
                    else:
                        value_for_key = author_1
                        value_for_value = author_2
                        
                    # Теперь - проверяем на согласные
                    first_difference_author_1 = get_first_different_character_for_a(author_1, author_2) # i
                    first_difference_author_2 = get_first_different_character_for_b(author_1, author_2) # u
                    result_of_comparison = is_two_characters_one_type(first_difference_author_1, first_difference_author_2)
                    type_1 = is_vowel(first_difference_author_1)
                    type_2 = is_vowel(first_difference_author_2)
                    
                    # Case 1: отличающиеся символы - гласные (высокая уверенность);
                    if result_of_comparison == True and type_1 == True:
                        if value_for_key not in group_1_dict:
                            group_1_dict[value_for_key] = [value_for_value]
                        else:
                            group_1_dict[value_for_key].append(value_for_value)
                    # Case 2: отличающиеися символы - согласные (средняя уверенность)
                    elif result_of_comparison == True and type_1 == False:
                        if value_for_key not in group_2_dict:
                            group_2_dict[value_for_key] = [value_for_value]
                        else:
                            group_2_dict[value_for_key].append(value_for_value)
                    # Case 3: отличающиеся символы различаются по типу, НО 'golosenko, i.', 'golossenko, i.',
                    # Case 4: отличающиеся символы отличаются по типу (гласная и согласная)
                    else:
                        # проверяем символы на дублирование
                        if type_2 == False:
                            index_of_first_dif_litera = author_2.index(first_difference_author_2) # берем индекс первого отличающегося звука
                            index_of_next_litera = index_of_first_dif_litera + 1
                            # Подстрока author_2[index_of_next_litera] в author_2 дублируется -> group_3_dict
                            if author_2[index_of_first_dif_litera] == author_2[index_of_next_litera]:
                                if value_for_key not in group_3_dict:
                                    group_3_dict[value_for_key] = [value_for_value]
                                else:
                                    group_3_dict[value_for_key].append(value_for_value)
                            # Подстрока не дублируется -> group_4_dict
                            else:
                                if value_for_key not in group_4_dict:
                                    group_4_dict[value_for_key] = [value_for_value]
                                else:
                                    group_4_dict[value_for_key].append(value_for_value)
                        elif type_1 == False:
                            index_of_first_dif_litera = author_1.index(first_difference_author_1) # берем индекс первого отличающегося звука
                            index_of_next_litera = index_of_first_dif_litera + 1
                            # Подстрока author_1[index_of_next_litera] в author_1 дублируется.
                            if author_1[index_of_first_dif_litera] == author_1[index_of_next_litera]:
                                if value_for_key not in group_3_dict:
                                    group_3_dict[value_for_key] = [value_for_value]
                                else:
                                    group_3_dict[value_for_key].append(value_for_value)
                            # Подстрока не дублируется -> group_4_dict
                            else:
                                if value_for_key not in group_4_dict:
                                    group_4_dict[value_for_key] = [value_for_value]
                                else:
                                    group_4_dict[value_for_key].append(value_for_value)

                    try:
                        author_1_list.remove(author_2)
                    except:
                        pass
                    try:
                        author_2_list.remove(author_1)
                    except:
                        pass
                    try:
                        author_2_list.remove(author_2)
                    except:
                        pass

 98%|██████████████████████████████████████▍| 8394/8523 [01:34<00:01, 88.84it/s]


In [337]:
group_1_dict = {k:list(set(v)) for k,v in group_1_dict.items()} # группа 1 - различия в гласных
group_1_dict

{'arutunian, y. v.': ['arutiunian, y. v.'],
 'beliaev, a. m.': ['belyaev, a. m.'],
 'boutenko, i. a.': ['butenko, i. a.'],
 'chaplaev, h. g.': ['chaplayev, h. g.'],
 'cherviakov, v. v.': ['chervyakov, v. v.'],
 'titayev, k. d.': ['titaev, k.'],
 'driakhlov, n. i.': ['dryakhlov, n. i.'],
 'dvojmenniy, i. a.': ['dvoimenniy, i. a.'],
 'faizullin, f. s.': ['fayzullin, f. s. z'],
 'gavriliuk, t. v.': ['gavrilyuk, t.'],
 'gishkayeva, l. l.': ['gishkaeva, l.'],
 'grigorieva, i. a.': ['grigoryeva, i.'],
 'iarskaya-smirnova, e. r.': ['iarskaia-smirnova, e. r.'],
 'iarskaia-smirnova, e. r.': ['iarskaya-smirnova, e. r.'],
 'kultygin, v. p.': ['kultigin, v. p.'],
 'krishtanovskaya, o. v.': ['kryshtanovskaya, o. v.'],
 'kulikova, n. v.': ['kulakova, n. v.'],
 'leontieva, e. o.': ['leontyeva, e.'],
 'magun, v. s.': ['magoun, v.'],
 'mersiyanova, i. v.': ['mersianova, i. v.'],
 'nikulin, a. m.': ['nikoulin, a. m.'],
 'podvoiskiy, d. g.': ['podvoyskiy, d. g.', 'podvoisky, d. g.'],
 'priamikova, e. v.'

In [338]:
group_2_dict = {k:list(set(v)) for k,v in group_2_dict.items()} # группа 2 - различия в cогласных
group_2_dict

{'bestuzhev-lada, i. v.': ['bestuzhevlada, i. v.'],
 'inglehardt, r. f.': ['inglehart, r. f.'],
 'katchanov, y. l.': ['kachanov, y.', 'katshanov, y. l.'],
 'netchiporenko, o. v.': ['netshiporenko, o. v.'],
 'plahov, v. d.': ['plakhov, v.'],
 'poluhina, e. v.': ['polukhina, e.'],
 'rukavichnikov, v. o.': ['rukavishnikov, v. o.'],
 'ryazantsev, i. p.': ['ryazantcev, i. p.'],
 'schulz, v. l.': ['shulz, v. l.'],
 'volchkova, l. t.': ['voltchkova, l. t.']}

In [339]:
group_3_dict = {k:list(set(v)) for k,v in group_3_dict.items()} # группа 3 - различия в дублирующих звуках
group_3_dict

{'denisenko, m. b.': ['denissenko, m. b.']}

In [340]:
group_4_dict = {k:list(set(v)) for k,v in group_4_dict.items()} # группа 4 - различия в гласных-согласных
group_4_dict

{'altukhov, s.': ['altukov, s.'],
 'avraamova, e. m.': ['avramova, e. m.'],
 'baimbetov, a. a.': ['bambetov, a. a.'],
 'bazanov, v. a.': ['bazhanov, v. a.'],
 'belenkiy, v. k.': ['belenkly, v. k.'],
 'bobkov, v. n.': ['boikov, v.'],
 'borisov, v. a.': ['bortsov, v. a.'],
 'bourganov, a. k.': ['bourghanov, a. k.'],
 'braslavskiy, r. g.': ['braslayskiy, r.'],
 'budanova, l. y.': ['buyanova, l. y.'],
 'cherdnichenko, g. a.': ['cherednichenko, g. a.'],
 'danovski, s. l.': ['danovskii, s. l.'],
 'dorogkin, j. n.': ['dorojkin, j. n.'],
 'drobiizheva, l. m.': ['drobizheva, l. m.'],
 'dvoimenniy, i. a.': ['dvoimennty, i. a.'],
 'dzhutsev, k. v.': ['dzutsev, k. v.'],
 'filonovitch, s. r.': ['filontovitch, s. r.'],
 'gendin, a. m.': ['ghendin, a. m.'],
 'gertchikov, v. i.': ['ghertchikov, v. i.'],
 'ghilmanov, a. z.': ['gilmanov, a. z.'],
 'gorbunov, a. v.': ['goryunov, a. v.'],
 'ighitkhanian, e. d.': ['igitkhanian, e. d.'],
 'kraeva, o. l.': ['karaeva, o.'],
 'karghina, i. g.': ['kargina, i. g

In [342]:
print(f'До обработки: {last_names.shape[0]} уникальных фамилий;')
print(f'До обработки: {names_and_patronymics.shape[0]} уникальных комбинаций имен-отчеств;\n\n')

df = change_author_names_with_dictionary(df, dict_for_change=group_1_dict, column_name='author')
df = change_author_names_with_dictionary(df, dict_for_change=group_2_dict, column_name='author')
df = change_author_names_with_dictionary(df, dict_for_change=group_3_dict, column_name='author')

last_names = get_unique_last_names(df, fullname_column='author', index_starts=1)
names_and_patronymics = get_unique_first_names_and_patronymics_combinations(df, fullname_column='author', index_starts=1)
print()
print(f'После обработки обработки: {last_names.shape[0]} уникальных фамилий;')
print(f'После обработки: {names_and_patronymics.shape[0]} уникальных комбинаций имен-отчеств;')

До обработки: 5488 уникальных фамилий;
До обработки: 508 уникальных комбинаций имен-отчеств;


Было произведено 85 замен.
Было произведено 26 замен.
Было произведено 2 замен.

После обработки обработки: 5450 уникальных фамилий;
После обработки: 507 уникальных комбинаций имен-отчеств;


In [343]:
df['author'].unique().shape[0]

7167

In [344]:
# код для поиска совпадений

group_1_dict = {} # гласные - vowel
group_2_dict = {} # согласные - consonant
group_3_dict = {} # дублирующие звуки
group_4_dict = {} # разные звуки - гласные/согласные

author_1_list = [i for i in df['author'].tolist() if ', ' in i]
author_2_list = [i for i in df['author'].tolist() if ', ' in i]

# идем по авторам
for author_1 in tqdm.tqdm(author_1_list):
    # если у нас длина фамилии не меньше 4
    if len(author_1.split(', ')[0]) > 4:
        # данные автора 1
        last_name_1 = author_1.split(', ')[0]
        initials_1 = author_1.split(', ')[1].split()
        #print(last_name_1)
        for author_2 in author_2_list:
            ## данные автора_2
            last_name_2 = author_2.split(', ')[0]
            initials_2 = author_2.split(', ')[1].split()
            # Если у нас длина фамилии не меньше 4 И ПРИ ЭТОМ окончания у фамилий одинаковые
            # И при этом первая буква тоже одинаковая (иначе - )
            # и при этом это не одни и те же строки (в точности; author_1 != author_2)
            # Почему важно? Род; чтобы мы случайно не заметчили "abalakina" и "abalakin"
            # И расстояние левенштейна == 1
            if len(author_2.split(', ')[0]) > 4 and\
                last_name_1[-1] == last_name_2[-1] and\
                ((last_name_1[0] == last_name_2[0]) | ((last_name_1.startswith('ch') and last_name_2.startswith('tch')) or (last_name_1.startswith('tch') and last_name_2.startswith('ch'))) | (last_name_1[0] in ['k', 'c'] and last_name_2[0] in ['k', 'c'])) and\
                author_1 != author_2 and\
                lev(last_name_1, last_name_2) == 1:
                # И расстояние левенштейна 1
                # И у нас одинаковые инициалы или совпадает первый инициал ->
                # записываем в словарь
                # этот способ позволяет не мэтчить авторов вслепую, а уделить больше внимания инициалам, не оставлять их без внимания

                lev_distance = lev(last_name_1, last_name_2)
                flag_1 = len(initials_1) == 2 # в имени первого автора два инициала? True/False
                flag_2 = len(initials_2) == 2 # в имени второго автора два инициала? True/False
                flag_3 = False # совпадают ли инициалы в достаточной мере?

                ## Case 1: два инициала у каждого имени (то есть, и имя, и отчество) или только один -> да, меняем
                if sum([flag_1, flag_2]) in [0, 2]:
                    if initials_1 == initials_2:
                        flag_3 = True 
        
                ## Case 2: (два инициала у первого и один у второго) ИЛИ (один инициал у второго)
                else:
                    ### пример: aleksandrov, m. a. И aleksandrov, m.
                    if initials_1[0] == initials_2[0] or sorted([initials_1[0], initials_2[0]]) == ['h', 'k']:
                        flag_3 = True
                        
                # Если все ок, обновляем словарь
                if flag_3 == True:
                    if (flag_1 < flag_2):
                        value_for_key = author_2
                        value_for_value = author_1
                    else:
                        value_for_key = author_1
                        value_for_value = author_2
                        
                    # Теперь - проверяем на согласные
                    first_difference_author_1 = get_first_different_character_for_a(author_1, author_2) # i
                    first_difference_author_2 = get_first_different_character_for_b(author_1, author_2) # u
                    result_of_comparison = is_two_characters_one_type(first_difference_author_1, first_difference_author_2)
                    type_1 = is_vowel(first_difference_author_1)
                    type_2 = is_vowel(first_difference_author_2)
                    
                    # Case 1: отличающиеся символы - гласные (высокая уверенность);
                    if result_of_comparison == True and type_1 == True:
                        if value_for_key not in group_1_dict:
                            group_1_dict[value_for_key] = [value_for_value]
                        else:
                            group_1_dict[value_for_key].append(value_for_value)
                    # Case 2: отличающиеися символы - согласные (средняя уверенность)
                    elif result_of_comparison == True and type_1 == False:
                        if value_for_key not in group_2_dict:
                            group_2_dict[value_for_key] = [value_for_value]
                        else:
                            group_2_dict[value_for_key].append(value_for_value)
                    # Case 3: отличающиеся символы различаются по типу, НО 'golosenko, i.', 'golossenko, i.',
                    # Case 4: отличающиеся символы отличаются по типу (гласная и согласная)
                    else:
                        # проверяем символы на дублирование
                        if type_2 == False:
                            index_of_first_dif_litera = author_2.index(first_difference_author_2) # берем индекс первого отличающегося звука
                            index_of_next_litera = index_of_first_dif_litera + 1
                            # Подстрока author_2[index_of_next_litera] в author_2 дублируется -> group_3_dict
                            if author_2[index_of_first_dif_litera] == author_2[index_of_next_litera]:
                                if value_for_key not in group_3_dict:
                                    group_3_dict[value_for_key] = [value_for_value]
                                else:
                                    group_3_dict[value_for_key].append(value_for_value)
                            # Подстрока не дублируется -> group_4_dict
                            else:
                                if value_for_key not in group_4_dict:
                                    group_4_dict[value_for_key] = [value_for_value]
                                else:
                                    group_4_dict[value_for_key].append(value_for_value)
                        elif type_1 == False:
                            index_of_first_dif_litera = author_1.index(first_difference_author_1) # берем индекс первого отличающегося звука
                            index_of_next_litera = index_of_first_dif_litera + 1
                            # Подстрока author_1[index_of_next_litera] в author_1 дублируется.
                            if author_1[index_of_first_dif_litera] == author_1[index_of_next_litera]:
                                if value_for_key not in group_3_dict:
                                    group_3_dict[value_for_key] = [value_for_value]
                                else:
                                    group_3_dict[value_for_key].append(value_for_value)
                            # Подстрока не дублируется -> group_4_dict
                            else:
                                if value_for_key not in group_4_dict:
                                    group_4_dict[value_for_key] = [value_for_value]
                                else:
                                    group_4_dict[value_for_key].append(value_for_value)

                    try:
                        author_1_list.remove(author_2)
                    except:
                        pass
                    try:
                        author_2_list.remove(author_1)
                    except:
                        pass
                    try:
                        author_2_list.remove(author_2)
                    except:
                        pass

 99%|██████████████████████████████████████▋| 8453/8523 [01:29<00:00, 94.54it/s]


In [345]:
group_1_dict = {k:list(set(v)) for k,v in group_1_dict.items()} # группа 1 - различия в гласных
group_1_dict

{}

In [346]:
group_2_dict = {k:list(set(v)) for k,v in group_2_dict.items()} # группа 1 - различия в гласных
group_2_dict

{}

In [347]:
group_3_dict = {k:list(set(v)) for k,v in group_3_dict.items()} # группа 1 - различия в гласных
group_3_dict

{}

In [348]:
group_4_dict = {k:list(set(v)) for k,v in group_4_dict.items()} # группа 1 - различия в гласных
group_4_dict

{'altukhov, s.': ['altukov, s.'],
 'avraamova, e. m.': ['avramova, e. m.'],
 'baimbetov, a. a.': ['bambetov, a. a.'],
 'bazanov, v. a.': ['bazhanov, v. a.'],
 'belenkiy, v. k.': ['belenkly, v. k.'],
 'bobkov, v. n.': ['boikov, v.'],
 'borisov, v. a.': ['bortsov, v. a.'],
 'bourganov, a. k.': ['bourghanov, a. k.'],
 'braslavskiy, r. g.': ['braslayskiy, r.'],
 'budanova, l. y.': ['buyanova, l. y.'],
 'cherdnichenko, g. a.': ['cherednichenko, g. a.'],
 'danovski, s. l.': ['danovskii, s. l.'],
 'dorogkin, j. n.': ['dorojkin, j. n.'],
 'drobiizheva, l. m.': ['drobizheva, l. m.'],
 'dvoimenniy, i. a.': ['dvoimennty, i. a.'],
 'dzhutsev, k. v.': ['dzutsev, k. v.'],
 'filonovitch, s. r.': ['filontovitch, s. r.'],
 'gendin, a. m.': ['ghendin, a. m.'],
 'gertchikov, v. i.': ['ghertchikov, v. i.'],
 'ghilmanov, a. z.': ['gilmanov, a. z.'],
 'gorbunov, a. v.': ['goryunov, a. v.'],
 'ighitkhanian, e. d.': ['igitkhanian, e. d.'],
 'kraeva, o. l.': ['karaeva, o.'],
 'karghina, i. g.': ['kargina, i. g

In [349]:
df

,author,documents,citations,total link strength,author_old_name,n_of_initials
1,"abaeva, a. m.",1,1,4,"abaeva, a. m.",2
2,"abaeva, a. m.",1,0,4,"abaeva, asiyat m.",2
3,"abaeva, o. p.",2,0,8,"abaeva, o. p.",2
4,"abalakin, a. a.",1,0,3,"abalakin, a. a.",2
5,"abalakina, t. v.",1,0,3,"abalakina, t. v.",2
...,...,...,...,...,...,...
8533,"zyryanov, v. v.",1,0,3,"zyrianov, v. v.",2
8534,"zyryanov, v. v.",2,3,3,"zyryanov, v. v.",2
8535,"zyryanov, v. v.",1,0,2,"zyryanov, vladimir v.",2
8536,"zyuganov, g. a.",1,0,20,"zyuganov, ga",2


In [366]:
df.drop(axis=1, columns=['n_of_occurancy'], inplace=True)

In [368]:
df.to_excel('final_data.xlsx')

In [ ]:
df['author'].count()

In [360]:
n_stage_1

7579

In [364]:
(7579-7167)/8537 * 100

4.826051306079418

In [352]:
(8537-7167)/8537

0.160477919643903

In [351]:
df['author'].unique().shape[0]

7167

In [288]:
# ВНИМАНИЕ - total link strength я везде проставила как 0
authors_list = df['author'].unique().tolist()

for author in tqdm.tqdm(authors_list):
    one_author_df = df[df['author'] == author]
    one_author_df['documents'] = one_author_df['documents'].sum()
    one_author_df['citations'] = one_author_df['citations'].sum()
    one_author_df['total link strength'] = 0
    one_author_df = one_author_df.iloc[0, :]
    df = pd.concat([df, one_author_df])
    df = df.drop_duplicates('author', keep='last')

['abaeva, a. m.',
 'abaeva, o. p.',
 'abalakin, a. a.',
 'abalakina, t. v.',
 'abalkin, l. i.',
 'abashidze, a. k.',
 'abasov, z. a.',
 'abassov, z. a.',
 'abazova, m. m.',
 'abdrashitov, e. y.',
 'abdrashitov, e. e.',
 'sadulaev, b. a.',
 'abdulaeva, e. s.',
 'abdulaeva, m. i.',
 'abdulagatov, z. m.',
 'abdulazimova, t. k.',
 'abdulgalimov, a. m.',
 'abdulhozhaev, i. s.',
 'abdulvahabova, b. b.',
 'abdulvahabova, l. b.',
 'abdurakhmanova, e. m.',
 'abgadzhava, d. a.',
 'ablagey, a. m.',
 'ablazhej, a. m.',
 'ablazhey, a. m.',
 'ablazhey, n. n.',
 'abramov, a. p.',
 'abramov, r. n.',
 'abramov, r.',
 'abramov, v. f.',
 'abramov, v. n.',
 'abramova, i. e.',
 'abramova, m. a.',
 'abramova, m. g.',
 'abramova, s. b.',
 'abrossimova, l. v.',
 'abroukov, v. s.',
 'abubakarov, m. v.',
 'abubakarova, m. a.',
 'abubakarova, m. m.',
 'abylaiuly, a. ..',
 'abylkalikov, s. i.',
 'adama, a.',
 'adamiantz, t. z.',
 'adamovich, k.',
 'adamyants, t. ..',
 'adamyants, t. z.',
 'adediran, a. m.',
 'ade

In [369]:
n_stage_2 = df['author'].unique().shape[0] # сколько авторов уникальных после второго этапа

**Post Scriptum**

Хотя выше была произведена подробная обработка данных, остаются нерешенные на данном этапе проблемы.

1. Есть авторы, имена и фамилии которых упомянуты в некорректной форме, но её нельзя привести к корректной `254	"andriyashinym, nikitoy"`; 
2. Авторы, у которых слишком. много. точек...... в имени; я не знаю, что с ними делать;
3. При использовании расстояния Левенштейна для определения наиболее схожих имен встречаются проблемные случаи: `'goryunov, a. v.': ['gorbunov, a.']`. Возможно, нужно как-то выяснитьь, какие символы отличаются (гласные/согласные)? Я пока не знаю такого решения

И, в конце концов, есть еще один альтернативный вариант решения проблемы, связанной с положением имен, отчеств и фамилий – есть библиотека Natasha для русского языка и альтернативные решения на английском, в процессе использования которой можно вычленить отчества, имена и фамилии; эта мысль пришла мне в голову уже ближе к концу и изначально данные были относительно нормально структурированы в этом вопросе, но 

In [18]:
# пункт 2
counter = 0

for index in range(df['author'].shape[0]):
    fullname = df.loc[index, 'author'] # тут у нас имя автора;
    if ',' in fullname:
        fullname_list = fullname.split(',') # тут у нас полный список: фамилия, имя отчество
        first_part_of_name = fullname_list[0]
        if fullname.count('.') > 2:
            print(fullname)
            counter += 1

print(f'\nОбнаружено {counter} таких проблемных случаев.')

boronina,  l. n. i.
glinskaya, m. l. f.
grigoriev,  s. i. v.
gudman, s. d. z.
hoebe, christian j. p. a.
ibragimov, m. -r. a.
toshchenko, zh. t. v. a.
vasilieva, l. n. i. r.

Обнаружено 8 таких проблемных случаев.
